In [33]:
from test_model import TestModel
import torch

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = TestModel().to(device)
x = torch.randn(16, 2).to(device)

In [34]:
out_fp32 = model(x)

In [35]:
def print_param_dtype(model):
    for name, param in model.named_parameters():
        print(f"{name} is loaded in {param.dtype}")

In [36]:
print(f'{out_fp32.shape}\n')

print_param_dtype(model)
print(out_fp32)

torch.Size([16, 8])

linear1.weight is loaded in torch.float32
linear1.bias is loaded in torch.float32
bn1.weight is loaded in torch.float32
bn1.bias is loaded in torch.float32
linear2.weight is loaded in torch.float32
linear2.bias is loaded in torch.float32
bn2.weight is loaded in torch.float32
bn2.bias is loaded in torch.float32
tensor([[5.6570e-01, 2.6057e-01, 6.0959e-02, 0.0000e+00, 0.0000e+00, 6.4186e-01,
         1.0861e+00, 7.2191e-01],
        [0.0000e+00, 0.0000e+00, 3.4649e-01, 1.0093e+00, 6.2260e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 3.4649e-01, 1.0093e+00, 6.2260e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.5711e-01, 1.4652e+00, 0.0000e+00, 0.0000e+00, 8.1230e-01, 9.1862e-01,
         0.0000e+00, 0.0000e+00],
        [1.9353e+00, 1.6048e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0986e+00,
         2.4531e+00, 1.3174e+00],
        [6.1562e-01, 0.0000e+00, 9.0805e-01, 0.0000e+00, 0.0000e+00, 6.0938e-01,
         1.

### Cast test model into float16 (Half).
- Outputs of the model are random, and change every time, even with a set seed. On such small values, the low-precision attribute of float16 bakes in rounding errors through multiple layers; in this case, batch norm, ReLU (removes negative values), linear. This generates different model outputs every run, in contrast to float32. float16 is also not supposed on some CPU kernels -> not an ideal data type in many cases.

In [56]:
model_fp16 = TestModel().to(dtype = torch.float16, device = device) # or TestModel().half()
x = x.to(torch.float16)

In [58]:
print_param_dtype(model_fp16)
out_fp16 = model_fp16(x)
print(out_fp16)

linear1.weight is loaded in torch.float16
linear1.bias is loaded in torch.float16
bn1.weight is loaded in torch.float16
bn1.bias is loaded in torch.float16
linear2.weight is loaded in torch.float16
linear2.bias is loaded in torch.float16
bn2.weight is loaded in torch.float16
bn2.bias is loaded in torch.float16
tensor([[0.0000, 0.1044, 0.0000, 0.0000, 0.9800, 0.0000, 0.7773, 0.0000],
        [0.7139, 0.0000, 0.4041, 0.0000, 0.0000, 0.0000, 0.0000, 0.7212],
        [0.7827, 0.0000, 0.7378, 0.0000, 0.0000, 0.0000, 0.0000, 1.1035],
        [0.0000, 0.7319, 1.1123, 1.7871, 0.1461, 1.7012, 0.0000, 0.9907],
        [0.0000, 0.0000, 0.0000, 0.1715, 1.3926, 0.2062, 0.4624, 0.0000],
        [0.6343, 0.0000, 0.0000, 0.0000, 0.4487, 0.0000, 1.3262, 0.0000],
        [0.5684, 0.0000, 0.0000, 0.0000, 0.5239, 0.0000, 1.2363, 0.0000],
        [0.4905, 0.0000, 0.0000, 0.0000, 0.6133, 0.0000, 1.1309, 0.0000],
        [0.0703, 1.0273, 0.0000, 0.3337, 0.0000, 0.4602, 0.0000, 0.2517],
        [0.5620, 0.000

### Cast test model into bfloat16.
- bfloat16 is stable, unlike float16, and achieves the same outputs after each run with a set seed.

In [30]:
from copy import deepcopy
model_bf16 = deepcopy(model)
model_bf16 = model_bf16.to(torch.bfloat16)
print_param_dtype(model_bf16)
out_bf16 = model_bf16(x.to(torch.bfloat16))
print(out_bf16)

linear1.weight is loaded in torch.bfloat16
linear1.bias is loaded in torch.bfloat16
bn1.weight is loaded in torch.bfloat16
bn1.bias is loaded in torch.bfloat16
linear2.weight is loaded in torch.bfloat16
linear2.bias is loaded in torch.bfloat16
bn2.weight is loaded in torch.bfloat16
bn2.bias is loaded in torch.bfloat16
tensor([[0.5703, 0.2402, 0.0532, 0.0000, 0.0000, 0.6211, 1.0781, 0.7109],
        [0.0000, 0.0000, 0.3457, 1.0000, 0.6211, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3457, 1.0000, 0.6211, 0.0000, 0.0000, 0.0000],
        [0.9531, 1.4609, 0.0000, 0.0000, 0.8086, 0.9141, 0.0000, 0.0000],
        [1.9375, 1.6094, 0.0000, 0.0000, 0.0000, 2.0938, 2.4375, 1.3203],
        [0.6328, 0.0000, 0.9062, 0.0000, 0.0000, 0.6055, 1.5000, 1.8203],
        [0.0000, 0.0000, 0.8516, 0.0000, 0.0000, 0.0000, 0.2178, 0.9492],
        [0.0000, 0.0000, 0.7578, 0.0000, 0.0000, 0.0000, 0.0000, 0.5469],
        [0.0000, 0.0000, 0.3320, 0.9961, 0.6211, 0.0000, 0.0000, 0.0000],
        [0.507

### Compute output differences between fp32 and bf16

In [31]:
mean_diff = torch.abs(out_bf16 - out_fp32).mean().item()
max_diff = torch.abs(out_bf16 - out_fp32).max().item()

print(f"Mean diff: {mean_diff} | Max diff: {max_diff}")

Mean diff: 0.00309927249327302 | Max diff: 0.029530048370361328


### Output logits hold similar values, small differences between the full-precision model and the bf16 model.
- Does not lead to a huge performance degradation, even on large models.